# Exemplo de um workflow de aprendizagem máquina

### *Conjunto de dados Human Activity Recognition using Smartphones*

Descrição do dataset: 
https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones

*The experiments have been carried out with a group of 30 volunteers (…). Each person performed six activities (WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING) wearing a smartphone (…). Using its embedded accelerometer and gyroscope, we captured 3-axial linear acceleration and 3-axial angular velocity (…). The experiments have been video-recorded to label the data manually. The dataset has been randomly partitioned into two sets, where 70% of the volunteers was selected for generating the training data and 30% the test data*

Descarregar os dados do link seguinte e descompactar o ZIP: https://archive.ics.uci.edu/ml/machine-learning-databases/00240/

**Estrutura dos dados** (ficheiros principais):
* Códigos das atividades: “activity_labels.txt” (2 colunas)
* Atributos: “features.txt” (561 linhas, 2 colunas)
* Indivíduos (treino ; teste): “train/subject_train.txt” (7352 linhas), “test/subject_test.txt” (2947 linhas), ambos com 1 coluna 
* Atributos de entrada – X (treino; teste): “train/X_train.txt” (7352 linhas), “test/X_test.txt” (2947 linhas), ambos com 561 colunas
* Atributo de saída (atividade) – y (treino; teste): “train/y_train.txt” (7352 linhas), “test/y_test.txt” (2947 linhas), ambos com 1 coluna


**Variáveis:**
For each record in the dataset it is provided: 
* A 561-feature vector with time and frequency domain variables. 
* Its activity label. 
* An identifier of the subject who carried out the experiment.


### Carregar os dados

Ao descompactar o ficheiro a pasta base será UCI HAR Dataset. Definir a variável folder abaixo com path absoluto dessa pasta. Se estiver na pasta onde está o notebook bastará: `folder = "./UCI HAR Dataset/"`

In [ ]:
folder = "./UCI HAR Dataset/"

In [ ]:
import pandas as pd

In [ ]:
activities = pd.read_csv(folder+'/activity_labels.txt', sep=' ', header=None, names=('ID','Activity'))
print(activities)

In [ ]:
features = pd.read_csv(folder+"/features.txt", sep = " ", header = None, names=('ID','Sensor'))
print(features.shape)
features.head()

In [ ]:
subjects_tr = pd.read_csv(folder+"/train/subject_train.txt", header = None, names=['SubjectID'])
subjects_tr.head()

In [ ]:
subjects_tst = pd.read_csv(folder+"/test/subject_test.txt", header = None, names=['SubjectID'])
print(subjects_tr.shape, subjects_tst.shape)

In [ ]:
x_train = pd.read_csv(folder+"/train/X_train.txt", sep = "\s+", header = None)
x_test = pd.read_csv(folder+"/test/X_test.txt", sep = "\s+", header = None)
print(x_train.shape, x_test.shape)

In [ ]:
y_train = pd.read_csv(folder+"/train/y_train.txt", header=None, names=['ActivityID'])
y_test = pd.read_csv(folder+"/test/y_test.txt", header=None, names=['ActivityID'])
print(y_train.shape, y_test.shape)

### Preparação dos dados

Juntar os conjuntos de dados de treino e teste

In [ ]:
subjects_all = pd.concat([subjects_tr, subjects_tst], ignore_index=True)
print(subjects_all.shape)

In [ ]:
x_all = pd.concat([x_train, x_test], ignore_index = True)
print(x_all.shape)

In [ ]:
y_all = y_train.append(y_test, ignore_index=True)
print(y_all.shape)

Colocar nomes das colunas de X como nomes das features

In [ ]:
sensorNames = features['Sensor']
x_all.columns = sensorNames
x_all.head()

Substituir códigos de atividade pela designação (string)

In [ ]:
for i in activities['ID']:
    activity = activities[activities['ID'] == i]['Activity'] 
    y_all = y_all.replace({i: activity.iloc[0]})
    
y_all.columns = ['Activity']
y_all.head()

In [ ]:
y_all.tail()

Juntar tudo num único DataFrame e guardar num CSV

In [ ]:
x_all = pd.concat([x_all, subjects_all], axis=1)
allXy = pd.concat([x_all, y_all], axis=1)
print(allXy.shape)

allXy.to_csv("HAR_clean.csv")

Agregação dos dados para um dataset mais pequeno (por indivíduo e por atividade)

In [ ]:
import numpy as np
grouped = allXy.groupby (['SubjectID', 'Activity']).aggregate(np.mean)

print(grouped.shape)
grouped.head()

grouped.to_csv("HAR_grouped.csv")

### Exploração do conjunto de dados

Caraterizar as distribuições de valores das variáveis de entrada

In [ ]:
input_data = allXy.iloc[:,:-2]

input_data.describe()

Caraterizar a distribuição de valores da variável de saída

In [ ]:
output_data = allXy.iloc[:,-1]
output_data.describe()

In [ ]:
allXy.groupby("Activity").size()

Verificar se existem valores nulos

In [ ]:
allXy.isnull().sum().sum()

Standardizar os dados

In [ ]:
from sklearn import preprocessing
sc_input = preprocessing.scale(input_data)

### Análise não supervisionada

Realizar um processo de PCA que explique pelo menos 80% da variabilidade

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.8)
pca.fit(sc_input)
X_reduced = pca.transform(sc_input)
X_reduced.shape[1]

Represente a variância explicada por cada uma das primeiras 10 PCs usando um gráfico apropriado

In [ ]:
import matplotlib.pyplot as plt

print('Var. explained: %s'% str(pca.explained_variance_ratio_))
n_pc = 10

plt.bar(range(n_pc), pca.explained_variance_ratio_[0:10]*100)
plt.xticks(range(n_pc), ['PC'+str(i) for i in range(1,n_pc+1)])
plt.title("Explained variance")
plt.ylabel("Percentage")
plt.show()

Construir scores plot com os resultados do PCA e comparar com variável *Activity*

In [ ]:
for act in allXy['Activity'].unique():
    sp = allXy.index[allXy['Activity']==act]-1
    plt.plot(X_reduced[sp,0],X_reduced[sp,1],'o',label=act)
plt.title("PCA")
plt.legend(loc='best', shadow=False)
plt.show()

Correr clustering k-means; comparar clusters com variável *Activity*

In [ ]:
from sklearn.cluster import KMeans

k=6
kmeans_har = KMeans(n_clusters=k, max_iter=1000)
kmeans_har.fit(sc_input)
labels = kmeans_har.labels_

pd.crosstab(labels, allXy["Activity"], rownames=['clusters'] )

Correr clustering hierárquico usando os dados agregados e visualizar árvore resultante colorindo folhas com variável *Activity*

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

grouped_sc = preprocessing.scale(grouped.iloc[:,2:])

Z = linkage(grouped_sc, method='single', metric='euclidean')

plt.figure(figsize=(25, 10))
dendrogram(Z, 
	labels=list(grouped.index.get_level_values(1)),    		leaf_rotation=90., leaf_font_size=8.)
plt.title('Hierarchical Clustering Dendrogram')
plt.ylabel('distance')
lcolors = {'STANDING':'b', "WALKING_UPSTAIRS":"m", "LAYING":'g', 'SITTING':'c', "WALKING" :"y", "WALKING_DOWNSTAIRS":"r"}
ax = plt.gca()
xlbls = ax.get_xmajorticklabels()
for lbl in xlbls:
  lbl.set_color(lcolors[lbl.get_text()])
plt.show()


### Aprendizagem máquina - modelos supervisionados

Dividir dados em partição de treino e teste (mantendo 30% no test set); verificar distribuição nas labels no training e test set

Treinar modelos de base com vários classificadores no conjunto de dados de treino. Avaliar cada um destes modelos com validação cruzada.

Considerando o modelo mais prometedor, experimentar a seleção de atributos baseadas em testes estatísticos univariados (ANOVA), considerando uma redução para metade do número de variáveis.

Considerando o modelo mais prometedor fazer um processo de otimização de hiperparâmetros

Criar um modelo ensemble com base nos 3 melhores modelos que experimentou

Estime o erro do melhor modelo obtido no test set.

Treinar o modelo final da forma mais adequada